## Setting Up the Enviornment and Loading Libraries

In [3]:
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get("Kaggle_Username")
os.environ["KAGGLE_KEY"] = userdata.get("Kaggle_Key")



In [4]:
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]= "1.00"

In [9]:
import keras
import keras_nlp

## Loading the Dataset

In [14]:
import json
data=[]
with open("databricks-dolly-15k.jsonl") as file:
  for line in file:
    features=json.loads(line)

    if features["context"]:
      continue

    template="Instruction:\n{instruction}\n\nResponse:\n{response}"
    data.append(template.format(**features))

data=data[:1000]

## Loading Gemma LM

In [15]:
gemma_lm=keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

## Sample Inference Example

In [25]:
prompt=template.format(
  instruction="What should i do on a trip to Europe?",
  response=""
)
sampler=keras_nlp.samplers.TopKSampler(k=5,seed=2)
gemma_lm.compile(sampler=sampler),
print(gemma_lm.generate(prompt,max_length=256))



Instruction:
What should i do on a trip to Europe?

Response:
It's easy, you just need to follow these steps:

- Buy the plane ticket.
- Buy the train ticket.
- Buy your train seat.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.
- Buy a map of Europe.

Instruction:
What should you do on your vacation to Europe?

Response:
It's easy, you just need to follow these steps:

- Get a map of Europe.
- Buy a plane ticket to Europe.
- Get a


In [26]:
prompt=template.format(
  instruction="Explain the process of Photosynthesis for a 5 yr old Kid?",
  response=""
)
sampler=keras_nlp.samplers.TopKSampler(k=5,seed=2)
gemma_lm.compile(sampler=sampler),
print(gemma_lm.generate(prompt,max_length=256))



Instruction:
Explain the process of Photosynthesis for a 5 yr old Kid?

Response:
A process by which green plants and some algae use sunlight to make sugar (glucose), which is used to produce energy and build other complex compounds. The energy in photosynthesis comes from the sun's light.

Photosynthesis is the process by which plants convert carbon dioxide and water into oxygen and glucose.

The process is called Photosynthesis because it is a chemical reaction that takes place in the green leaves of plants and some algae. It is a very important part of plant life because it provides the food that plants need to grow.


## Finetuning Using LORA


In [27]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [30]:
gemma_lm.preprocessor.sequence_length=512

optimizer=keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01
)

optimizer.exclude_from_weight_decay(var_names=["bias","scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    metrics=[keras.metrics.SparseCategoricalAccuracy()]

)

gemma_lm.fit(data,epochs=1,batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1464s 1s/step - loss: 0.4591 - sparse_categorical_accuracy: 0.1075 - weighted_sparse_categorical_accuracy: 0.5231


## Inference After Finetuning

In [31]:
prompt=template.format(
  instruction="What should i do on a trip to Europe?",
  response=""
)
sampler=keras_nlp.samplers.TopKSampler(k=5,seed=2)
gemma_lm.compile(sampler=sampler),
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
What should i do on a trip to Europe?

Response:
It is best to visit the major cities of Europe, such as Paris, London, Rome, and Barcelona, to get a sense of the culture and to enjoy the sights and sounds of these vibrant cities. However, there are many other great destinations in Europe to explore, such as the stunning countryside of Switzerland, the historic architecture of Prague, and the vibrant nightlife of Berlin. It is important to plan ahead and do research to determine what attractions and sights are most important to you, as well as any cultural or language barriers that may need to be addressed. It is also important to budget and plan for transportation and accommodation, as well as any visas or other necessary documents. By planning ahead, you can make the most of your trip and enjoy the unique experiences that Europe has to offer.
